In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_info_columns', 999)

In [3]:
PLOT_LOCS = '../data/processed/blm_usfs_wadnr_plot_footprints.shp'
CLIMATE = '../data/processed/pnw_lidar_plots_climatewna_2000-2017MSY.csv'
CLOUDMETS = '../data/processed/lidar/plot_clips/cloudmetrics.csv'
GRIDSURF = '../data/processed/lidar/plot_clips/gridsurfacestats.csv'
TOPOMETS = '../data/processed/lidar/plot_clips/topometrics.csv'
SOILS = '../data/processed/soils/soils_data_forPlots.csv'
POTVEG = '../data/processed/pot_veg/pot_veg_plots.csv'
WATER = '../data/interim/dist_to_water/water_distance_fromPlots.csv'
LANDSAT = '../data/interim/landsat_metrics.csv'
BLM_FVS = '../data/interim/fvs_outputs/BLM_plot_chars.csv'
USFS_FVS = '../data/interim/fvs_outputs/USFS_plot_chars.csv'
DNR_FVS = '../data/interim/fvs_outputs/WADNR_plot_chars.csv'

In [4]:
plot_locs = gpd.read_file(PLOT_LOCS)
plot_locs['uuid_part'] = plot_locs['uuid'].apply(lambda x: x.split('-')[0])

In [5]:
uuid_lookup = plot_locs[['uuid', 'uuid_part']].set_index('uuid_part')

In [6]:
plot_locs.head()

,comments,lat,lon,meas_date,orig_id,plot_id,source,meas_yr,uuid,geometry,uuid_part
0,None,41.995946,-120.106540,None,60200000000000,None,USFS-FREMONT-WINEMA,2018,d7c01e3a-38e0-4bc2-a69c-7d5a204e2663,POLYGON ((-120.1063249094551 41.99594429160435...,d7c01e3a
1,None,41.995967,-120.089987,None,60200000000000,None,USFS-FREMONT-WINEMA,2018,c16be14e-f913-4516-9c4b-078b3d71371d,POLYGON ((-120.0897712098171 41.99596580702698...,c16be14e
2,None,42.014315,-120.180756,None,60200000000000,None,USFS-FREMONT-WINEMA,2018,b4e059b8-6f08-4d33-8c3c-2b0410d2a226,POLYGON ((-120.1805406386182 42.01431325658718...,b4e059b8
3,None,42.020315,-120.254986,None,60200000000000,None,USFS-FREMONT-WINEMA,2018,e3b77390-f724-4a58-ad90-0b97cb138ce8,POLYGON ((-120.2547708144149 42.02031362163422...,e3b77390
4,None,42.020516,-120.172500,None,60200000000000,None,USFS-FREMONT-WINEMA,2018,5588b367-b5dc-4b23-9af3-bcf0d6e2a571,POLYGON ((-120.1722846155213 42.02051432932783...,5588b367


In [7]:
climate = pd.read_csv(CLIMATE).rename(
    {'Year': 'year', 'ID1':'uuid_part', 'ID2':'source'},
    axis=1)

In [8]:
climate['uuid'] = uuid_lookup.loc[climate.uuid_part.values].values[:,0]
climate = climate.set_index(['uuid', 'year'])

In [9]:
# breaking the climatena columns into groups
MONTHLY_COLS = ['Tmax01','Tmax02','Tmax03','Tmax04','Tmax05','Tmax06',
                'Tmax07','Tmax08','Tmax09','Tmax10','Tmax11','Tmax12',
                'Tmin01','Tmin02','Tmin03','Tmin04','Tmin05','Tmin06',
                'Tmin07','Tmin08','Tmin09','Tmin10','Tmin11','Tmin12',
                'Tave01','Tave02','Tave03','Tave04','Tave05','Tave06',
                'Tave07','Tave08','Tave09','Tave10','Tave11','Tave12',
                'PPT01','PPT02','PPT03','PPT04','PPT05','PPT06',
                'PPT07','PPT08','PPT09','PPT10','PPT11','PPT12',
                'Rad01','Rad02','Rad03','Rad04','Rad05','Rad06',
                'Rad07','Rad08','Rad09','Rad10','Rad11','Rad12',
                'DD_0_01','DD_0_02','DD_0_03','DD_0_04','DD_0_05','DD_0_06',
                'DD_0_07','DD_0_08','DD_0_09','DD_0_10','DD_0_11','DD_0_12',
                'DD5_01','DD5_02','DD5_03','DD5_04','DD5_05','DD5_06',
                'DD5_07','DD5_08','DD5_09','DD5_10','DD5_11','DD5_12',
                'DD_18_01','DD_18_02','DD_18_03','DD_18_04','DD_18_05','DD_18_06',
                'DD_18_07','DD_18_08','DD_18_09','DD_18_10','DD_18_11','DD_18_12',
                'DD18_01','DD18_02','DD18_03','DD18_04','DD18_05','DD18_06',
                'DD18_07','DD18_08','DD18_09','DD18_10','DD18_11','DD18_12',
                'NFFD01','NFFD02','NFFD03','NFFD04','NFFD05','NFFD06',
                'NFFD07','NFFD08','NFFD09','NFFD10','NFFD11','NFFD12',
                'PAS01','PAS02','PAS03','PAS04','PAS05','PAS06',
                'PAS07','PAS08','PAS09','PAS10','PAS11','PAS12',
                'Eref01','Eref02','Eref03','Eref04','Eref05','Eref06',
                'Eref07','Eref08','Eref09','Eref10','Eref11','Eref12',
                'CMD01','CMD02','CMD03','CMD04','CMD05','CMD06',
                'CMD07','CMD08','CMD09','CMD10','CMD11','CMD12',
                'RH01','RH02','RH03','RH04','RH05','RH06',
                'RH07','RH08','RH09','RH10','RH11','RH12']
SEASON_COLS = ['Tmax_wt','Tmax_sp','Tmax_sm','Tmax_at',
               'Tmin_wt','Tmin_sp','Tmin_sm','Tmin_at',
               'Tave_wt','Tave_sp','Tave_sm','Tave_at',
               'PPT_wt','PPT_sp','PPT_sm','PPT_at',
               'Rad_wt','Rad_sp','Rad_sm','Rad_at',
               'DD_0_wt','DD_0_sp','DD_0_sm','DD_0_at',
               'DD5_wt','DD5_sp','DD5_sm','DD5_at',
               'DD_18_wt','DD_18_sp','DD_18_sm','DD_18_at',
               'DD18_wt','DD18_sp','DD18_sm','DD18_at',
               'NFFD_wt','NFFD_sp','NFFD_sm','NFFD_at',
               'PAS_wt','PAS_sp','PAS_sm','PAS_at',
               'Eref_wt','Eref_sp','Eref_sm','Eref_at',
               'CMD_wt','CMD_sp','CMD_sm','CMD_at',
               'RH_wt','RH_sp','RH_sm','RH_at']
ANN_COLS = ['MAT','MWMT','MCMT','TD','MAP','MSP',
            'AHM','SHM','DD_0','DD5','DD_18','DD18',
            'NFFD','bFFP','eFFP','FFP','PAS','EMT',
            'EXT','MAR','Eref','CMD','RH']

In [10]:
climate_season = climate[SEASON_COLS].replace(-9999.0, np.nan).drop(['Rad_wt','Rad_sp','Rad_sm','Rad_at'], axis=1)
climate_annual = climate[ANN_COLS].replace(-9999.0, np.nan).drop('MAR', axis=1)
climate_season.columns = [col.lower() for col in climate_season.columns]
climate_annual.columns = [col.lower() for col in climate_annual.columns]

In [11]:
climate_annual.head()

,,mat,mwmt,mcmt,td,map,msp,ahm,shm,dd_0,dd5,dd_18,dd18,nffd,bffp,effp,ffp,pas,emt,ext,eref,cmd,rh
uuid,year,,,,,,,,,,,,,,,,,,,,,,
d7c01e3a-38e0-4bc2-a69c-7d5a204e2663,2000,5.0,15.4,-2.8,18.2,1019,119,14.7,129.9,542,1233,4733,30,172,156,261,105,365,-33.4,34.5,655,428,62
c16be14e-f913-4516-9c4b-078b3d71371d,2000,6.6,17.3,-1.3,18.6,680,89,24.5,194.2,395,1566,4199,71,184,151,267,116,159,-32.3,36.9,789,536,58
b4e059b8-6f08-4d33-8c3c-2b0410d2a226,2000,4.4,14.7,-3.2,17.9,1019,127,14.2,116.1,596,1106,4948,21,165,161,259,98,403,-33.9,33.9,625,395,63
e3b77390-f724-4a58-ad90-0b97cb138ce8,2000,7.0,17.7,-1.4,19.0,583,96,29.2,184.2,380,1666,4086,88,185,148,267,119,125,-33.2,37.5,811,551,56
5588b367-b5dc-4b23-9af3-bcf0d6e2a571,2000,4.5,14.8,-3.2,18.0,1033,128,14.0,115.1,596,1112,4940,22,167,159,260,101,407,-33.8,33.8,621,390,64


In [12]:
climate_annual.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 91602 entries, (d7c01e3a-38e0-4bc2-a69c-7d5a204e2663, 2000) to (4214cf54-1e3f-45d3-b93c-80c965cb3314, 2017)
Data columns (total 22 columns):
mat      91602 non-null float64
mwmt     91602 non-null float64
mcmt     91602 non-null float64
td       91602 non-null float64
map      91602 non-null int64
msp      91602 non-null int64
ahm      91602 non-null float64
shm      91602 non-null float64
dd_0     91602 non-null int64
dd5      91602 non-null int64
dd_18    91602 non-null int64
dd18     91602 non-null int64
nffd     91602 non-null int64
bffp     91602 non-null int64
effp     91602 non-null int64
ffp      91602 non-null int64
pas      91602 non-null int64
emt      91602 non-null float64
ext      91602 non-null float64
eref     91602 non-null int64
cmd      91602 non-null int64
rh       91602 non-null int64
dtypes: float64(8), int64(14)
memory usage: 15.7+ MB


In [13]:
climate_season.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 91602 entries, (d7c01e3a-38e0-4bc2-a69c-7d5a204e2663, 2000) to (4214cf54-1e3f-45d3-b93c-80c965cb3314, 2017)
Data columns (total 52 columns):
tmax_wt     91602 non-null float64
tmax_sp     91602 non-null float64
tmax_sm     91602 non-null float64
tmax_at     91602 non-null float64
tmin_wt     91602 non-null float64
tmin_sp     91602 non-null float64
tmin_sm     91602 non-null float64
tmin_at     91602 non-null float64
tave_wt     91602 non-null float64
tave_sp     91602 non-null float64
tave_sm     91602 non-null float64
tave_at     91602 non-null float64
ppt_wt      91602 non-null int64
ppt_sp      91602 non-null int64
ppt_sm      91602 non-null int64
ppt_at      91602 non-null int64
dd_0_wt     91602 non-null int64
dd_0_sp     91602 non-null int64
dd_0_sm     91602 non-null int64
dd_0_at     91602 non-null int64
dd5_wt      91602 non-null int64
dd5_sp      91602 non-null int64
dd5_sm      91602 non-null int64
dd5_at      91602 non-null

In [14]:
cloudmets = pd.read_csv(CLOUDMETS, low_memory=False)
cloudmets['uuid'] = cloudmets.FileTitle.apply(lambda x: x.split('_')[0])
cloudmets['year'] = cloudmets.FileTitle.apply(lambda x: int(x.split('_')[-1].split('-')[-1]))
cloudmets = cloudmets.set_index(['uuid', 'year'])

In [15]:
RENAME_COLS = {'Elev strata (below 0.15) return proportion':'strat0_return-proportion',
               'Elev strata (0.15 to 1.37) return proportion':'strat1_return-proportion',
               'Elev strata (5.00 to 10.00) return proportion':'strat2_return-proportion',
               'Elev strata (10.00 to 20.00) return proportion':'strat3_return-proportion',
               'Elev strata (20.00 to 30.00) return proportion':'strat4_return-proportion',
               'Elev strata (above 30.00) return proportion':'strat5_return-proportion',
               'Int strata (below 0.15) median':'strat0_intensity-median',
               'Int strata (0.15 to 1.37) median':'strat1_intensity-median',
               'Int strata (1.37 to 5.00) median':'strat2_intensity-median',
               'Int strata (5.00 to 10.00) median':'strat3_intensity-median',
               'Int strata (10.00 to 20.00) median':'strat4_intensity-median',
               'Int strata (above 30.00) median':'strat5_intensity-median',
               'Elev P05':'height_05-percentile','Elev P25':'height_25-percentile',
               'Elev P50':'height_50-percentile',
               'Elev P75':'height_75-percentile',
               'Elev P95':'height_95_percentile',
               'Elev maximum':'height_max',
               'Percentage all returns above 1.37':'cover'}

In [16]:
cloudmets = cloudmets.rename(RENAME_COLS, axis=1)
KEEP_COLS = [col for col in RENAME_COLS.values()]
cloudmets = cloudmets[KEEP_COLS]
cloudmets.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5135 entries, (aef73d5b-3e3d-4519-802f-a09c591a8931, 2014) to (80c46ebf-c0c0-4597-8fab-267573b28f91, 2014)
Data columns (total 19 columns):
strat0_return-proportion    5135 non-null float64
strat1_return-proportion    5135 non-null float64
strat2_return-proportion    5135 non-null float64
strat3_return-proportion    5135 non-null float64
strat4_return-proportion    5135 non-null float64
strat5_return-proportion    5135 non-null float64
strat0_intensity-median     5135 non-null float64
strat1_intensity-median     5135 non-null float64
strat2_intensity-median     5135 non-null float64
strat3_intensity-median     5135 non-null float64
strat4_intensity-median     5135 non-null float64
strat5_intensity-median     5135 non-null float64
height_05-percentile        5135 non-null float64
height_25-percentile        5135 non-null float64
height_50-percentile        5135 non-null float64
height_75-percentile        5135 non-null float64
height_95_

In [17]:
cloudmets.head()

,,strat0_return-proportion,strat1_return-proportion,strat2_return-proportion,strat3_return-proportion,strat4_return-proportion,strat5_return-proportion,strat0_intensity-median,strat1_intensity-median,strat2_intensity-median,strat3_intensity-median,strat4_intensity-median,strat5_intensity-median,height_05-percentile,height_25-percentile,height_50-percentile,height_75-percentile,height_95_percentile,height_max,cover
uuid,year,,,,,,,,,,,,,,,,,,,
aef73d5b-3e3d-4519-802f-a09c591a8931,2014,0.030641,0.061181,0.013088,0.011769,0.015219,0.850244,41.5,45.0,64.0,55.0,78.5,82.0,0.4600,50.754997,78.820000,96.002502,110.462502,121.699997,90.817776
0f9c587f-efbc-49f5-8123-1912356bd73e,2006,0.023585,0.022013,0.000000,0.001572,0.001572,0.940252,8.0,12.5,5.0,-9999.0,-9999.0,14.0,2.0625,82.737503,99.854996,110.522499,120.052505,122.000000,95.440252
4ca85db2-ca72-4426-b53c-702421c3177c,2006,0.060647,0.203136,0.034060,0.219677,0.125582,0.283631,26.0,26.0,22.0,19.0,20.0,17.0,0.0000,1.190000,15.520000,32.474998,55.609501,79.860001,73.621661
b88e67f8-0f86-491a-bfbe-26196e0400b6,2002,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
600fdbb4-d4d1-462f-af11-8221ce698376,2006,0.309091,0.490909,0.044406,0.085664,0.040210,0.008042,31.0,28.0,30.0,27.0,26.0,30.0,0.0000,0.000000,0.420000,0.830000,19.513000,37.590000,20.000000


In [18]:
gridsurf = pd.read_csv(GRIDSURF)
gridsurf['uuid'] = gridsurf.plot_id.apply(lambda x: x.split('_')[0])
gridsurf['year'] = gridsurf.plot_id.apply(lambda x: int(x.split('_')[-1].split('-')[-1]))
gridsurf = gridsurf.set_index(['uuid', 'year'])
KEEP_COLS = ['potential_volume', 'stddev_height',
             'surface_area_ratio', 'surface_volume', 'surface_volume_ratio']
gridsurf = gridsurf[KEEP_COLS]
gridsurf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5135 entries, (00027724-39d2-42b3-954d-cf2dbe527a44, 2015) to (fff7e1c3-5bb4-4eaf-b92c-ed417421aaf3, 2008)
Data columns (total 5 columns):
potential_volume        5135 non-null float64
stddev_height           5135 non-null float64
surface_area_ratio      5135 non-null float64
surface_volume          5135 non-null float64
surface_volume_ratio    4830 non-null float64
dtypes: float64(5)
memory usage: 249.3+ KB


In [19]:
gridsurf.head()

potential_volume  stddev_height  \
uuid                                 year                                    
00027724-39d2-42b3-954d-cf2dbe527a44 2015              94.0       0.079546   
0080963b-a3ff-4081-be77-02eb7b458d30 2008            5046.0      11.256773   
0083eb5e-aed6-4c22-9412-bc116d072ed9 2006            2658.0       8.932764   
008dc9d1-d2a0-4ee5-ba75-592318d06c1b 2013            3718.5       2.579358   
                                     2016            3895.0       2.444401   

                                           surface_area_ratio  surface_volume  \
uuid                                 year                                       
00027724-39d2-42b3-954d-cf2dbe527a44 2015            1.024375        1.406838   
0080963b-a3ff-4081-be77-02eb7b458d30 2008            7.121909     3391.452148   
0083eb5e-aed6-4c22-9412-bc116d072ed9 2006            6.092628      831.681885   
008dc9d1-d2a0-4ee5-ba75-592318d06c1b 2013            2.883042     3173.707031   
                                     2016            2.553517     3310.491699   

                                           surface_volume_ratio  
uuid                                 year                        
00027724-39d2-42b3-954d-cf2dbe527a44 2015              0.025674  
0080963b-a3ff-4081-be77-02eb7b458d30 2008              0.717014  
0083eb5e-aed6-4c22-9412-bc116d072ed9 2006              0.312898  
008dc9d1-d2a0-4ee5-ba75-592318d06c1b 2013              0.860225  
                                     2016              0.869114

In [20]:
topomets = pd.read_csv(TOPOMETS)
topomets['uuid'] = topomets.plot_id.apply(lambda x: x.split('_')[0])
topomets['year'] = topomets.plot_id.apply(lambda x: int(x.split('_')[-1].split('-')[-1]))
topomets = topomets.set_index(['uuid', 'year']).drop('plot_id', axis=1)
topomets.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5135 entries, (00027724-39d2-42b3-954d-cf2dbe527a44, 2015) to (fff7e1c3-5bb4-4eaf-b92c-ed417421aaf3, 2008)
Data columns (total 7 columns):
aspect                   5069 non-null float64
elevation                5135 non-null float64
overall_curvature        5069 non-null float64
plan_curvature           5069 non-null float64
profile_curvature        5069 non-null float64
slope                    5069 non-null float64
solar_radiation_index    5069 non-null float64
dtypes: float64(7)
memory usage: 329.5+ KB


In [21]:
soils = pd.read_csv(SOILS, index_col = 'uuid')
soils.columns = [col.lower() for col in soils.columns]
KEEP_SOILS = ['bulk_dens', 'soil_depth', 'pct_clay_surf', 'pct_rock_surf', 'pct_sand_surf']
soils = soils[KEEP_SOILS]
soils.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5089 entries, 075cbfbc-f210-49f9-9f4e-2133b16ac2ec to 25d94682-4fb4-466a-8345-851d5a0d7bbf
Data columns (total 5 columns):
bulk_dens        5089 non-null int64
soil_depth       5089 non-null float64
pct_clay_surf    5089 non-null int64
pct_rock_surf    5089 non-null int64
pct_sand_surf    5089 non-null int64
dtypes: float64(1), int64(4)
memory usage: 238.5+ KB


In [22]:
soils.head()

,bulk_dens,soil_depth,pct_clay_surf,pct_rock_surf,pct_sand_surf
uuid,,,,,
075cbfbc-f210-49f9-9f4e-2133b16ac2ec,135,140.0,9,24,48
ca06b2b7-5e93-453b-b31a-7411d57b0a31,92,127.0,13,24,42
3464eb42-56a2-486e-bd6a-39d9cc7d25f9,125,160.0,15,6,31
0c159329-d696-4cf1-a59d-a9b0dd9885a1,102,113.0,14,17,37
f6358940-c502-4128-a88e-38d0fa359d33,102,113.0,14,17,37


In [23]:
print(list(soils.columns))

['bulk_dens', 'soil_depth', 'pct_clay_surf', 'pct_rock_surf', 'pct_sand_surf']


In [24]:
potveg = pd.read_csv(POTVEG, index_col='uuid')
potveg.columns = [col.lower() for col in potveg.columns]
potveg = potveg.rename({'esp2':'pot_veg_type'}, axis=1)
potveg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5089 entries, 0fc9bef5-f82b-4af9-932a-8c064a339807 to 25d94682-4fb4-466a-8345-851d5a0d7bbf
Data columns (total 8 columns):
value           5089 non-null int64
oid_            5089 non-null int64
mz              5089 non-null int64
zone_name       5089 non-null object
pot_veg_type    5089 non-null int64
esp_name        5089 non-null object
esp_lf          5089 non-null int64
esplf_name      5089 non-null object
dtypes: int64(5), object(3)
memory usage: 357.8+ KB


In [25]:
potveg.head()

,value,oid_,mz,zone_name,pot_veg_type,esp_name,esp_lf,esplf_name
uuid,,,,,,,,
0fc9bef5-f82b-4af9-932a-8c064a339807,549,408,1,Northern Cascades,1178,North Pacific Hypermaritime Western Red-cedar-...,5,Upland Forest
e3f1682b-aeb2-4bf6-a20a-9f34775f9f86,97,97,1,Northern Cascades,1039,North Pacific Maritime Mesic-Wet Douglas-fir-W...,5,Upland Forest
f9fbe1a4-427a-4fc7-81ea-c73a64c6444d,97,97,1,Northern Cascades,1039,North Pacific Maritime Mesic-Wet Douglas-fir-W...,5,Upland Forest
a8b9dc65-264c-4878-a057-f6a7d748cef4,77,77,1,Northern Cascades,1037,North Pacific Maritime Dry-Mesic Douglas-fir-W...,5,Upland Forest
f70c4a18-6312-4039-8ff2-263626a62d68,549,408,1,Northern Cascades,1178,North Pacific Hypermaritime Western Red-cedar-...,5,Upland Forest


In [26]:
landsat = pd.read_csv(LANDSAT, index_col=['uuid', 'year']).drop('source', axis=1)
landsat.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 86513 entries, (f646c521-31b0-4567-898c-d4a27422b5c2, 2002) to (2ea15f7d-f4f5-4125-aff8-1f5c72b3b169, 2018)
Data columns (total 12 columns):
green         82471 non-null float64
blue          82471 non-null float64
red           82471 non-null float64
nir           82471 non-null float64
endvi         82471 non-null float64
ndvi          82471 non-null float64
savi          82471 non-null float64
swir1         82471 non-null float64
swir2         82471 non-null float64
wetness       82471 non-null float64
greenness     82471 non-null float64
brightness    82471 non-null float64
dtypes: float64(12)
memory usage: 8.2+ MB


In [27]:
landsat.head()

,,green,blue,red,nir,endvi,ndvi,savi,swir1,swir2,wetness,greenness,brightness
uuid,year,,,,,,,,,,,,
f646c521-31b0-4567-898c-d4a27422b5c2,2002,471.693412,291.197852,414.169344,2128.877075,0.634134,0.675934,1.013700,1447.531182,712.798563,-106.251329,108.155288,235.320061
ce529ff1-a35d-4b97-ab44-5bf1c3b46731,2002,448.728468,281.513119,384.378287,1893.107702,0.612208,0.662206,0.993092,1261.750953,667.959718,-94.009805,93.630321,212.299320
75ff6bd4-88b3-46be-ae69-f2cfc6c18d04,2002,575.412711,339.793653,535.595238,2254.174572,0.612905,0.616818,0.925061,1757.976150,983.063477,-139.841463,104.347095,268.102042
78986eb4-cac2-4c81-866e-d1941e8180ab,2002,329.276805,192.131577,223.192984,1903.442521,0.706367,0.789872,1.184529,732.211490,324.636454,-40.079778,112.346049,176.469964
bc47424e-a736-4139-97b2-e2dbaee0ee4d,2002,626.951118,392.962073,614.204571,2370.380209,0.584216,0.588262,0.882244,2003.645721,1122.536913,-161.888053,104.353707,291.868012


In [28]:
water = pd.read_csv(WATER, index_col='uuid')
water.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5089 entries, 075cbfbc-f210-49f9-9f4e-2133b16ac2ec to 25d94682-4fb4-466a-8345-851d5a0d7bbf
Data columns (total 1 columns):
distance_to_water_m    5089 non-null float64
dtypes: float64(1)
memory usage: 79.5+ KB


In [29]:
water.head()

,distance_to_water_m
uuid,
075cbfbc-f210-49f9-9f4e-2133b16ac2ec,23.966350
ca06b2b7-5e93-453b-b31a-7411d57b0a31,249.961364
3464eb42-56a2-486e-bd6a-39d9cc7d25f9,382.598565
0c159329-d696-4cf1-a59d-a9b0dd9885a1,414.057799
f6358940-c502-4128-a88e-38d0fa359d33,110.280018


In [30]:
blm_fvs = pd.read_csv(BLM_FVS).set_index(['uuid', 'year'])
usfs_fvs = pd.read_csv(USFS_FVS).set_index(['uuid', 'year'])
dnr_fvs = pd.read_csv(DNR_FVS).set_index(['uuid', 'year'])

fvs = pd.concat([blm_fvs, usfs_fvs, dnr_fvs],
                sort=True,
                axis=0)
fvs = fvs.fillna(0)

cols = list(fvs.columns)
for col in ['structure_class', 'sdi', 'total_cover', 'qmd', 'ba', 'tpa']:
    cols.insert(0, cols.pop(cols.index(col)))

fvs = fvs.loc[:,cols].drop('standid', axis=1)

In [31]:
fvs.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 50267 entries, (f845c720-2870-40a9-a2ed-e0753806f50c, 2015) to (ffdb5b08-e27a-4d02-832e-1ce92b8fa75e, 2023)
Data columns (total 49 columns):
tpa                50267 non-null float64
ba                 50267 non-null float64
qmd                50267 non-null float64
total_cover        50267 non-null float64
sdi                50267 non-null float64
structure_class    50267 non-null object
AF                 50267 non-null float64
AS                 50267 non-null float64
BM                 50267 non-null float64
BO                 50267 non-null float64
CH                 50267 non-null float64
CW                 50267 non-null float64
DF                 50267 non-null float64
DG                 50267 non-null float64
ES                 50267 non-null float64
GC                 50267 non-null float64
GF                 50267 non-null float64
IC                 50267 non-null float64
JP                 50267 non-null float64
KP          

In [32]:
JOIN_DFS = [water, cloudmets, gridsurf, topomets, soils, landsat, potveg[['pot_veg_type']]] 
joined_x_annual = climate_annual.join(JOIN_DFS)
joined_x_seasonal = climate_season.join(JOIN_DFS)

In [33]:
for df in JOIN_DFS:
    joined_x_annual.dropna(subset=df.columns, how='all', inplace=True)
    joined_x_seasonal.dropna(subset=df.columns, how='all', inplace=True)

In [34]:
joined_x_annual.to_csv('../data/processed/features_annual.csv')
joined_x_annual.to_csv('../data/processed/features_seasonal.csv')

In [35]:
big_join = joined_x_annual.reset_index().merge(fvs.reset_index(), on='uuid', suffixes=('','_fvs'))

In [36]:
big_join['year_diff'] = abs(big_join['year_fvs'] - big_join['year'])

In [37]:
joined_data = big_join.loc[big_join.year_diff <= 10].drop('year_fvs', axis=1)

In [38]:
joined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37911 entries, 0 to 52389
Data columns (total 124 columns):
uuid                        37911 non-null object
year                        37911 non-null int64
mat                         37911 non-null float64
mwmt                        37911 non-null float64
mcmt                        37911 non-null float64
td                          37911 non-null float64
map                         37911 non-null int64
msp                         37911 non-null int64
ahm                         37911 non-null float64
shm                         37911 non-null float64
dd_0                        37911 non-null int64
dd5                         37911 non-null int64
dd_18                       37911 non-null int64
dd18                        37911 non-null int64
nffd                        37911 non-null int64
bffp                        37911 non-null int64
effp                        37911 non-null int64
ffp                         37911 non-null i

In [39]:
spp_codes = [x for x in joined_data.columns if str.isupper(x)]
sum_ba_prop = joined_data[spp_codes].sum(axis=1)
for sp in spp_codes:
    joined_data[sp] = joined_data[sp] / sum_ba_prop

In [40]:
# convert from individual tree species to FIA species groups
FVS_SPP = '../data/external/FVS_WesternSpeciesCrosswalk.csv'
fvs_spp = pd.read_csv(FVS_SPP)

fvs_spp = fvs_spp.dropna(subset=['FIA CODE (3 digit)']).rename(
    {'FIA CODE (3 digit)': 'SPCD',
     'USDA PLANTS SYMBOL': 'SYMBOL',
     'FVS ALPHA CODE': 'FVS_ALPHA',
     'COMMON NAME (from FSVeg)': 'COMMON_NAME',
     'SCIENTIFIC NAME (from FSVeg)': 'SCIENTIFIC_NAME'}, 
    axis=1)

fvs_spp['SPCD'] = fvs_spp['SPCD'].astype(int)

KEEP_COLS = ['SYMBOL', 'SPCD', 'FVS_ALPHA', 'COMMON_NAME', 'SCIENTIFIC_NAME']
fvs_spp = fvs_spp[KEEP_COLS].set_index('SPCD')
fvs_spp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 10 to 981
Data columns (total 4 columns):
SYMBOL             164 non-null object
FVS_ALPHA          165 non-null object
COMMON_NAME        166 non-null object
SCIENTIFIC_NAME    166 non-null object
dtypes: object(4)
memory usage: 6.5+ KB


In [41]:
# jenkins species group codes: description
jenk_codes = {
    1: 'Cedar/larch',
    2: 'Douglas-fir',
    3: 'True fir/hemlock',
    4: 'Pine',
    5: 'Spruce',
    6: 'Aspen/alder/cottonwood-willow',
    7: 'Soft maple/birch',
    8: 'Mixed hardwood',
    9: 'Hard maple/oak/hickory/beech',
    10: 'Juniper/oak/mesquite'}

In [42]:
FIA_SPP = '../data/external/PNWFIA_REF_SPECIES.csv'
fia_spp = pd.read_csv(FIA_SPP)

KEEP_COLS = ['SPCD', 'W_SPGRPCD', 'SPP_GRP_NAME', 'JENKINS_SPGRPCD', 'CLASS']
fia_spp = fia_spp[KEEP_COLS].set_index('SPCD')
fia_spp['SPP_GRP_NAME'] = fia_spp['SPP_GRP_NAME'].str.replace(' ', '_').str.upper()
fia_spp['CLASS'] = fia_spp['CLASS'].str.upper()
fia_spp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455 entries, 10 to 999
Data columns (total 4 columns):
W_SPGRPCD          455 non-null int64
SPP_GRP_NAME       455 non-null object
JENKINS_SPGRPCD    455 non-null int64
CLASS              455 non-null object
dtypes: int64(2), object(2)
memory usage: 17.8+ KB


In [43]:
spp_lookup = fvs_spp.merge(fia_spp, left_index=True, right_index=True).reset_index().set_index('FVS_ALPHA')

In [44]:
spp_lookup.loc['OS', 'SPP_GRP_NAME'] = 'OTHER_WESTERN_SOFTWOODS'
spp_lookup.loc['OS', 'CLASS'] = 'SOFTWOOD'
spp_lookup['SPP_GRP_NAME'] = spp_lookup['SPP_GRP_NAME'].str.replace('ZOTHER', 'OTHER')
spp_lookup['SPP_GRP_NAME'] = spp_lookup['SPP_GRP_NAME'].str.replace('_', '-')

In [45]:
spp_lookup.head()

,SPCD,SYMBOL,COMMON_NAME,SCIENTIFIC_NAME,W_SPGRPCD,SPP_GRP_NAME,JENKINS_SPGRPCD,CLASS
FVS_ALPHA,,,,,,,,
OF,10.0,ABIES,fir species,Abies,12.0,TRUE-FIR,3.0,SOFTWOOD
SH,21.0,ABSH,Shasta red fir,Abies ×shastensis,12.0,TRUE-FIR,3.0,SOFTWOOD
SF,11.0,ABAM,Pacific silver fir,Abies amabilis,12.0,TRUE-FIR,3.0,SOFTWOOD
SL,14.0,ABBR,bristlecone fir,Abies bracteata,12.0,TRUE-FIR,3.0,SOFTWOOD
WF,15.0,ABCO,white fir,Abies concolor,12.0,TRUE-FIR,3.0,SOFTWOOD


In [46]:
observed_spp_groups = spp_lookup.loc[spp_codes, 'SPP_GRP_NAME'].reset_index().set_index('SPP_GRP_NAME')
observed_spp_groups

,FVS_ALPHA
SPP_GRP_NAME,
TRUE-FIR,AF
COTTONWOOD-AND-ASPEN,AS
OTHER-WESTERN-HARDWOODS,BM
OAK,BO
OTHER-WESTERN-HARDWOODS,CH
COTTONWOOD-AND-ASPEN,CW
DOUGLAS-FIR,DF
OTHER-WESTERN-HARDWOODS,DG
ENGELMANN-AND-OTHER-SPRUCES,ES


In [47]:
observed_swhw = spp_lookup.loc[spp_codes, 'CLASS'].reset_index().set_index('CLASS')
observed_swhw.head()

,FVS_ALPHA
CLASS,
SOFTWOOD,AF
HARDWOOD,AS
HARDWOOD,BM
HARDWOOD,BO
HARDWOOD,CH


In [48]:
for spp_grp in pd.unique(observed_spp_groups.index):
    joined_data[spp_grp+'_baprop'] = (joined_data[list(observed_spp_groups.loc[spp_grp].values.flatten())].sum(axis=1)*100).astype(int)
    
for spp_class in pd.unique(observed_swhw.index):
    joined_data[spp_class+'_baprop'] = (joined_data[list(observed_swhw.loc[spp_class].values.flatten())].sum(axis=1)*100).astype(int)

In [49]:
joined_data.head()

,uuid,year,mat,mwmt,mcmt,td,map,msp,ahm,shm,dd_0,dd5,dd_18,dd18,nffd,bffp,effp,ffp,pas,emt,ext,eref,cmd,rh,distance_to_water_m,strat0_return-proportion,strat1_return-proportion,strat2_return-proportion,strat3_return-proportion,strat4_return-proportion,strat5_return-proportion,strat0_intensity-median,strat1_intensity-median,strat2_intensity-median,strat3_intensity-median,strat4_intensity-median,strat5_intensity-median,height_05-percentile,height_25-percentile,height_50-percentile,height_75-percentile,height_95_percentile,height_max,cover,potential_volume,stddev_height,surface_area_ratio,surface_volume,surface_volume_ratio,aspect,elevation,overall_curvature,plan_curvature,profile_curvature,slope,solar_radiation_index,bulk_dens,soil_depth,pct_clay_surf,pct_rock_surf,pct_sand_surf,green,blue,red,nir,endvi,ndvi,savi,swir1,swir2,wetness,greenness,brightness,pot_veg_type,tpa,ba,qmd,total_cover,sdi,structure_class,AF,AS,BM,BO,CH,CW,DF,DG,ES,GC,GF,IC,JP,KP,LO,LP,MA,MH,NF,OH,OS,OT,PC,PL,PP,PY,RA,RC,RF,SF,SP,SS,TO,WA,WB,WF,WH,WI,WJ,WL,WO,WP,YC,year_diff,TRUE-FIR_baprop,COTTONWOOD-AND-ASPEN_baprop,OTHER-WESTERN-HARDWOODS_baprop,OAK_baprop,DOUGLAS-FIR_baprop,ENGELMANN-AND-OTHER-SPRUCES_baprop,INCENSE-CEDAR_baprop,PONDEROSA-AND-JEFFREY-PINES_baprop,OTHER-WESTERN-SOFTWOODS_baprop,LODGEPOLE-PINE_baprop,RED-ALDER_baprop,WESTERN-REDCEDAR_baprop,SUGAR-PINE_baprop,SITKA-SPRUCE_baprop,WESTERN-HEMLOCK_baprop,WESTERN-LARCH_baprop,WESTERN-WHITE-PINE_baprop,SOFTWOOD_baprop,HARDWOOD_baprop
0,00027724-39d2-42b3-954d-cf2dbe527a44,2015,8.2,19.8,-2.5,22.4,435,104,41.8,191.1,357,2029,3744,182,192,140,259,118,98,-34.5,38.1,936,700,53,89.723536,0.911437,0.015141,0.038568,0.01038,0.0,0.0,91.0,99.0,55.0,61.0,86.0,-9999.0,0.0,0.0,0.0,0.0,4.83,12.91,7.342158,94.0,0.079546,1.024375,1.406838,0.025674,131.976303,1328.897217,0.487,-0.07015,0.10245,11.9352,1.7981,106,118.0,18,22,16,812.262438,549.739428,978.890547,2124.200871,0.458228,0.370753,0.556039,2304.209577,1702.590796,-153.266894,64.111872,345.627083,1159,8.0,10.0,15.31,5.0,16.0,0=BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,100,0
1,00027724-39d2-42b3-954d-cf2dbe527a44,2015,8.2,19.8,-2.5,22.4,435,104,41.8,191.1,357,2029,3744,182,192,140,259,118,98,-34.5,38.1,936,700,53,89.723536,0.911437,0.015141,0.038568,0.01038,0.0,0.0,91.0,99.0,55.0,61.0,86.0,-9999.0,0.0,0.0,0.0,0.0,4.83,12.91,7.342158,94.0,0.079546,1.024375,1.406838,0.025674,131.976303,1328.897217,0.487,-0.07015,0.10245,11.9352,1.7981,106,118.0,18,22,16,812.262438,549.739428,978.890547,2124.200871,0.458228,0.370753,0.556039,2304.209577,1702.590796,-153.266894,64.111872,345.627083,1159,8.0,10.0,15.57,5.0,16.0,0=BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,100,0
2,00027724-39d2-42b3-954d-cf2dbe527a44,2015,8.2,19.8,-2.5,22.4,435,104,41.8,191.1,357,2029,3744,182,192,140,259,118,98,-34.5,38.1,936,700,53,89.723536,0.911437,0.015141,0.038568,0.01038,0.0,0.0,91.0,99.0,55.0,61.0,86.0,-9999.0,0.0,0.0,0.0,0.0,4.83,12.91,7.342158,94.0,0.079546,1.024375,1.406838,0.025674,131.976303,1328.897217,0.487,-0.07015,0.10245,11.9352,1.7981,106,118.0,18,22,16,812.262438,549.739428,978.890547,2124.200871,0.458228,0.370753,0.556039,2304.209577,1702.590796,-153.266894,64.111872,345.627083,1159,8.0,11.0,15.82,5.0,17.0,0=BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,100,0
3,00027724-39d2-42b3-954d-cf2dbe527a44,2015,8.2,19.8,-2.5,22.4,435,104,41.8,191.1,357,2029,3744,182,192,140,259,118,98,-34.5,38.1,936,700,53,89.723536,0.911437,0.015141,0.038568,0.01038,0.0,0.0,91.0,99.0,55.0,61.0,86.0,-99

In [50]:
for sp in spp_codes:
    joined_data[sp + '_spp_abund_code'] = pd.cut(joined_data[sp], bins=[0, 0.33, 0.67, 1.0], right=True, include_lowest=True, labels=[0,1,2]).astype(int)

for spp_grp in pd.unique(observed_spp_groups.index):
    joined_data[spp_grp + '_sppgrp_abund_code'] = pd.cut(joined_data[spp_grp+'_baprop'], bins=[0, 33, 67, 100], right=True, include_lowest=True, labels=[0,1,2]).astype(int)
    
for spp_class in pd.unique(observed_swhw.index):
    joined_data[spp_class + '_sppclass_abund_code'] = pd.cut(joined_data[spp_class+'_baprop'], bins=[0, 50, 100], right=True, include_lowest=True, labels=[0,1]).astype(int)

# for sp in spp_codes:
#     joined_data[sp + '_bapa'] = joined_data['ba'] * joined_data[sp]
#     joined_data[sp + '_bapa'] = joined_data[sp + '_bapa'].fillna(0)
joined_data = joined_data.drop(spp_codes, axis=1)

In [51]:
spp_abund_codes = [x for x in joined_data.columns if '_spp_abund_code' in x]
sppgrp_abund_codes = [x for x in joined_data.columns if '_sppgrp_abund_code' in x]
sppclass_abund_codes = [x for x in joined_data.columns if '_sppclass_abund_code' in x]
baprops = [x for x in joined_data.columns if '_baprop' in x]

In [52]:
def get_spp_forest_type_name(row):
#     if (row[spp_abund_codes] == 2).any(axis='index'):
#         name = row[spp_abund_codes].index[row[spp_abund_codes] == 2][0].split('_spp_abund_code')[0]
    if (row[spp_abund_codes] >= 1).any(axis='index'):
        name = '_'.join([x.split('_spp_abund_code')[0] for x in row[spp_abund_codes].index[row[spp_abund_codes] >= 1]])  # + '_MIX'
    elif (row[sppclass_abund_codes] == 1).any(axis='index'):
        name = row[sppclass_abund_codes].index[row[sppclass_abund_codes] == 1][0].split('_sppclass_abund_code')[0] + '_MIX'
    else:
        name = 'NONSTOCKED'
    return name   

In [53]:
def get_sppgrp_forest_type_name(row):
#     if (row[sppgrp_abund_codes] == 2).any(axis='index'):
#         name = row[sppgrp_abund_codes].index[row[sppgrp_abund_codes] == 2][0].split('_sppgrp_abund_code')[0]
    if (row[sppgrp_abund_codes] >= 1).any(axis='index'):
        name = '_'.join([x.split('_sppgrp_abund_code')[0] for x in row[sppgrp_abund_codes].index[row[sppgrp_abund_codes] >= 1]])  # + '_MIX'
    elif (row[sppclass_abund_codes] == 1).any(axis='index'):
        name = row[sppclass_abund_codes].index[row[sppclass_abund_codes] == 1][0].split('_sppclass_abund_code')[0] + '_MIX'
    else:
        name = 'NONSTOCKED'
    return name    

In [54]:
joined_data['spp_forest_type_name'] = joined_data[spp_abund_codes + sppclass_abund_codes].apply(get_spp_forest_type_name, axis=1)
print('Done.')

Done.


In [55]:
joined_data['sppgrp_forest_type_name'] = joined_data[sppgrp_abund_codes + sppclass_abund_codes].apply(get_sppgrp_forest_type_name, axis=1)
print('Done.')

Done.


In [56]:
joined_data.loc[joined_data.sppgrp_forest_type_name == '']

,uuid,year,mat,mwmt,mcmt,td,map,msp,ahm,shm,dd_0,dd5,dd_18,dd18,nffd,bffp,effp,ffp,pas,emt,ext,eref,cmd,rh,distance_to_water_m,strat0_return-proportion,strat1_return-proportion,strat2_return-proportion,strat3_return-proportion,strat4_return-proportion,strat5_return-proportion,strat0_intensity-median,strat1_intensity-median,strat2_intensity-median,strat3_intensity-median,strat4_intensity-median,strat5_intensity-median,height_05-percentile,height_25-percentile,height_50-percentile,height_75-percentile,height_95_percentile,height_max,cover,potential_volume,stddev_height,surface_area_ratio,surface_volume,surface_volume_ratio,aspect,elevation,overall_curvature,plan_curvature,profile_curvature,slope,solar_radiation_index,bulk_dens,soil_depth,pct_clay_surf,pct_rock_surf,pct_sand_surf,green,blue,red,nir,endvi,ndvi,savi,swir1,swir2,wetness,greenness,brightness,pot_veg_type,tpa,ba,qmd,total_cover,sdi,structure_class,year_diff,TRUE-FIR_baprop,COTTONWOOD-AND-ASPEN_baprop,OTHER-WESTERN-HARDWOODS_baprop,OAK_baprop,DOUGLAS-FIR_baprop,ENGELMANN-AND-OTHER-SPRUCES_baprop,INCENSE-CEDAR_baprop,PONDEROSA-AND-JEFFREY-PINES_baprop,OTHER-WESTERN-SOFTWOODS_baprop,LODGEPOLE-PINE_baprop,RED-ALDER_baprop,WESTERN-REDCEDAR_baprop,SUGAR-PINE_baprop,SITKA-SPRUCE_baprop,WESTERN-HEMLOCK_baprop,WESTERN-LARCH_baprop,WESTERN-WHITE-PINE_baprop,SOFTWOOD_baprop,HARDWOOD_baprop,AF_spp_abund_code,AS_spp_abund_code,BM_spp_abund_code,BO_spp_abund_code,CH_spp_abund_code,CW_spp_abund_code,DF_spp_abund_code,DG_spp_abund_code,ES_spp_abund_code,GC_spp_abund_code,GF_spp_abund_code,IC_spp_abund_code,JP_spp_abund_code,KP_spp_abund_code,LO_spp_abund_code,LP_spp_abund_code,MA_spp_abund_code,MH_spp_abund_code,NF_spp_abund_code,OH_spp_abund_code,OS_spp_abund_code,OT_spp_abund_code,PC_spp_abund_code,PL_spp_abund_code,PP_spp_abund_code,PY_spp_abund_code,RA_spp_abund_code,RC_spp_abund_code,RF_spp_abund_code,SF_spp_abund_code,SP_spp_abund_code,SS_spp_abund_code,TO_spp_abund_code,WA_spp_abund_code,WB_spp_abund_code,WF_spp_abund_code,WH_spp_abund_code,WI_spp_abund_code,WJ_spp_abund_code,WL_spp_abund_code,WO_spp_abund_code,WP_spp_abund_code,YC_spp_abund_code,TRUE-FIR_sppgrp_abund_code,COTTONWOOD-AND-ASPEN_sppgrp_abund_code,OTHER-WESTERN-HARDWOODS_sppgrp_abund_code,OAK_sppgrp_abund_code,DOUGLAS-FIR_sppgrp_abund_code,ENGELMANN-AND-OTHER-SPRUCES_sppgrp_abund_code,INCENSE-CEDAR_sppgrp_abund_code,PONDEROSA-AND-JEFFREY-PINES_sppgrp_abund_code,OTHER-WESTERN-SOFTWOODS_sppgrp_abund_code,LODGEPOLE-PINE_sppgrp_abund_code,RED-ALDER_sppgrp_abund_code,WESTERN-REDCEDAR_sppgrp_abund_code,SUGAR-PINE_sppgrp_abund_code,SITKA-SPRUCE_sppgrp_abund_code,WESTERN-HEMLOCK_sppgrp_abund_code,WESTERN-LARCH_sppgrp_abund_code,WESTERN-WHITE-PINE_sppgrp_abund_code,SOFTWOOD_sppclass_abund_code,HARDWOOD_sppclass_abund_code,spp_forest_type_name,sppgrp_forest_type_name


In [57]:
joined_data = joined_data.drop(spp_abund_codes + sppgrp_abund_codes + sppclass_abund_codes + baprops, axis=1)

In [58]:
joined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37911 entries, 0 to 52389
Data columns (total 83 columns):
uuid                        37911 non-null object
year                        37911 non-null int64
mat                         37911 non-null float64
mwmt                        37911 non-null float64
mcmt                        37911 non-null float64
td                          37911 non-null float64
map                         37911 non-null int64
msp                         37911 non-null int64
ahm                         37911 non-null float64
shm                         37911 non-null float64
dd_0                        37911 non-null int64
dd5                         37911 non-null int64
dd_18                       37911 non-null int64
dd18                        37911 non-null int64
nffd                        37911 non-null int64
bffp                        37911 non-null int64
effp                        37911 non-null int64
ffp                         37911 non-null in

In [59]:
joined_data.to_csv('../data/processed/training_data_annual.csv', index=False)